In [1]:
import numpy as np
import matplotlib.pyplot as plt
import sys
import scipy
import scipy.optimize
import pyuvdata
import time

In [2]:
def get_test_data():

    model_path = '/Users/ruby/Astro/fhd_rlb_model_GLEAM_Aug2021'
    model_use_model = True
    data_path = '/Users/ruby/Astro/fhd_rlb_model_GLEAM_Aug2021'
    data_use_model = True
    obsid = '1061316296'
    pol = 'XX'

    model_filelist = ['{}/{}'.format(model_path, file) for file in [
        'vis_data/{}_vis_{}.sav'.format(obsid, pol),
        'vis_data/{}_vis_model_{}.sav'.format(obsid, pol),
        'vis_data/{}_flags.sav'.format(obsid),
        'metadata/{}_params.sav'.format(obsid),
        'metadata/{}_settings.txt'.format(obsid),
        'metadata/{}_layout.sav'.format(obsid)
    ]]
    data_filelist = ['{}/{}'.format(data_path, file) for file in [
        'vis_data/{}_vis_{}.sav'.format(obsid, pol),
        'vis_data/{}_vis_model_{}.sav'.format(obsid, pol),
        'vis_data/{}_flags.sav'.format(obsid),
        'metadata/{}_params.sav'.format(obsid),
        'metadata/{}_settings.txt'.format(obsid),
        'metadata/{}_layout.sav'.format(obsid)
    ]]

    model = pyuvdata.UVData()
    print('Reading model...')
    model.read_fhd(model_filelist, use_model=model_use_model)
    if data_path != model_path or model_use_model != data_use_model:
        data = pyuvdata.UVData()
        print('Reading data...')
        data.read_fhd(data_filelist, use_model=data_use_model)
        print('Done.')
    else:
        print('Using model for data')
        data = model.copy()

    # For testing, use one time and frequency only
    use_time = data.time_array[200000]
    #use_frequencies = data.freq_array[0, 100]
    #data.select(times=use_time, frequencies=use_frequencies)
    #model.select(times=use_time, frequencies=use_frequencies)
    data.select(times=use_time)
    model.select(times=use_time)

    # Need to check that the baseline ordering agrees between model and data

    return data, model

In [3]:
def initialize_cal(data):

    cal = pyuvdata.UVCal()
    cal.Nants_data = data.Nants_data
    cal.Nants_telescope = data.Nants_telescope
    cal.Nfreqs = data.Nfreqs
    cal.Njones = 1
    cal.Nspws = 1
    cal.Ntimes = 1
    cal.ant_array = np.intersect1d(data.ant_1_array, data.ant_2_array)
    cal.antenna_names = data.antenna_names
    cal.antenna_numbers = data.antenna_numbers
    cal.cal_style = 'sky'
    cal.cal_type = 'gain'
    cal.channel_width = data.channel_width
    cal.freq_array = data.freq_array
    cal.gain_convention = 'divide'
    cal.history = ''
    cal.integration_time = np.mean(data.integration_time)
    cal.jones_array = np.array([-5])
    cal.spw_array = data.spw_array
    cal.telescope_name = data.telescope_name
    cal.time_array = np.array([np.mean(data.time_array)])
    cal.x_orientation = 'east'
    cal.gain_array = np.full((
        cal.Nants_data, cal.Nspws, cal.Nfreqs, cal.Ntimes, cal.Njones
    ), 1, dtype=complex)
    cal.flag_array = np.full((
        cal.Nants_data, cal.Nspws, cal.Nfreqs, cal.Ntimes, cal.Njones
    ), False, dtype=bool)
    cal.quality_array = np.full((
        cal.Nants_data, cal.Nspws, cal.Nfreqs, cal.Ntimes, cal.Njones
    ), 1., dtype=float)
    cal.ref_antenna_name = ''
    cal.sky_catalog = 'GLEAM_bright_sources'
    cal.sky_field = 'phase center (RA, Dec): ({}, {})'.format(
        np.degrees(np.mean(data.phase_center_app_ra)),
        np.degrees(np.mean(data.phase_center_app_dec))
    )

    if not cal.check():
        print('ERROR: UVCal check failed.')
        sys.exit(1)

    return cal

In [4]:
data, model = get_test_data()

cal = initialize_cal(data)

# Create gains expand matrices
gains_exp_mat_1 = np.zeros((data.Nblts, cal.Nants_data), dtype=int)
gains_exp_mat_2 = np.zeros((data.Nblts, cal.Nants_data), dtype=int)
antenna_list = np.unique([data.ant_1_array, data.ant_2_array])
for baseline in range(data.Nblts):
    gains_exp_mat_1[
        baseline, np.where(antenna_list == data.ant_1_array[baseline])
    ] = 1
    gains_exp_mat_2[
        baseline, np.where(antenna_list == data.ant_2_array[baseline])
    ] = 1

# Initialize gains
gain_init_noise = .1
gains_init = (np.random.normal(
    1., gain_init_noise, size=(cal.Nants_data, cal.Nfreqs)
) + 1.j*np.random.normal(
    0., gain_init_noise, size=(cal.Nants_data, cal.Nfreqs)
))

# Define covariance matrix
cov_mat = np.identity(cal.Nfreqs)
cov_mat = np.repeat(cov_mat[np.newaxis, :, :], data.Nblts, axis=0)
print(np.shape(cov_mat))

# Ensure that the angle of the gains is mean-zero for each frequency
#avg_angle = np.arctan2(
#    np.mean(np.sin(np.angle(gains_fit)), axis=0),
#    np.mean(np.cos(np.angle(gains_fit)), axis=0)
#)
#gains_fit *= np.cos(avg_angle) - 1j*np.sin(avg_angle)

Reading model...


Telescope location derived from obs lat/lon/alt values does not match the location in the layout file. Using the value from known_telescopes.
tile_names from obs structure does not match antenna_names from layout


Using model for data
(8128, 384, 384)


In [5]:
gains = gains_init
model_vis = np.squeeze(model.data_array, axis=(1,3)) # Remove spw and pol axes
data_vis = np.squeeze(data.data_array, axis=(1,3))


In [6]:
def calculate_grad_real(gains_exp_mat_1, gains_exp_mat_2, gains, data_vis, model_vis, cov_mat):
    gains1_expanded = np.matmul(gains_exp_mat_1, gains)
    gains2_expanded = np.matmul(gains_exp_mat_2, gains)
    term1_part1 = -np.conj(gains1_expanded*model_vis)
    term2_part1 = -gains2_expanded*np.conj(model_vis)
    cost_term = data_vis - gains1_expanded*np.conj(gains2_expanded)*model_vis
    weighted_part2 = np.squeeze(np.matmul(cost_term[:, np.newaxis, :], cov_mat))
    term1 = np.matmul(gains_exp_mat_2.T, term1_part1*weighted_part2)
    term2 = np.matmul(gains_exp_mat_1.T, term2_part1*weighted_part2)
    grad = 2*np.real(term1 + term2)
    return grad

In [7]:
def calculate_grad_imag(gains_exp_mat_1, gains_exp_mat_2, gains, data_vis, model_vis, cov_mat):
    gains1_expanded = np.matmul(gains_exp_mat_1, gains)
    gains2_expanded = np.matmul(gains_exp_mat_2, gains)
    term1_part1 = -1.j*np.conj(gains1_expanded*model_vis)
    term2_part1 = 1.j*gains2_expanded*np.conj(model_vis)
    cost_term = data_vis - gains1_expanded*np.conj(gains2_expanded)*model_vis
    weighted_part2 = np.squeeze(np.matmul(cost_term[:, np.newaxis, :], cov_mat))
    term1 = np.matmul(gains_exp_mat_2.T, term1_part1*weighted_part2)
    term2 = np.matmul(gains_exp_mat_1.T, term2_part1*weighted_part2)
    grad = 2*np.real(term1 + term2)
    return grad

In [8]:
def calculate_negloglikelihood(gains_exp_mat_1, gains_exp_mat_2, gains, data_vis, model_vis, cov_mat):
    gains1_expanded = np.matmul(gains_exp_mat_1, gains)
    gains2_expanded = np.matmul(gains_exp_mat_2, gains)
    cost_term = data_vis - gains1_expanded*np.conj(gains2_expanded)*model_vis
    weighted_part2 = np.squeeze(np.matmul(cost_term[:, np.newaxis, :], cov_mat))
    negloglikelihood = np.real(np.sum(np.conj(cost_term)*weighted_part2))
    return negloglikelihood

In [9]:
# Test real gradient calculation
delta_gains = 0.0001
gains0 = np.copy(gains)
gains0[100, 200] -= delta_gains/2.
gains1 = np.copy(gains)
gains1[100, 200] += delta_gains/2.
negloglikelihood0 = calculate_negloglikelihood(gains_exp_mat_1, gains_exp_mat_2, gains0, data_vis, model_vis, cov_mat)
negloglikelihood1 = calculate_negloglikelihood(gains_exp_mat_1, gains_exp_mat_2, gains1, data_vis, model_vis, cov_mat)
grad_real = calculate_grad_real(gains_exp_mat_1, gains_exp_mat_2, gains, data_vis, model_vis, cov_mat)
print((negloglikelihood1-negloglikelihood0)/delta_gains)
print(grad_real[100, 200])

-555580.849647522
-555580.8577801876


In [10]:
# Test imaginary gradient calculation
delta_gains = 0.0001
gains0 = np.copy(gains)
gains0[100, 200] -= 1j*delta_gains/2.
gains1 = np.copy(gains)
gains1[100, 200] += 1j*delta_gains/2.
negloglikelihood0 = calculate_negloglikelihood(gains_exp_mat_1, gains_exp_mat_2, gains0, data_vis, model_vis, cov_mat)
negloglikelihood1 = calculate_negloglikelihood(gains_exp_mat_1, gains_exp_mat_2, gains1, data_vis, model_vis, cov_mat)
grad_imag = calculate_grad_imag(gains_exp_mat_1, gains_exp_mat_2, gains, data_vis, model_vis, cov_mat)
print((negloglikelihood1-negloglikelihood0)/delta_gains)
print(grad_imag[100, 200])

106899.2805480957
106899.27746565267


In [11]:
np.shape(cov_mat)

(8128, 384, 384)